# Create a X.509 certificate

- This is a Jupyter Notebook using Python 3.7 and openSSl to create a X.509 self-signed certificate for authenticating the signer using the openSSL command line tool.

- This will Generate RSA256 public and private keys for signing

- Typically you will this DO THIS ONLY ONCE and reuse the certificate.

*Although self-signed certificates are used for the purpose of these examples, they are not recommended for production systems.*

### Step 1: Pre-configure the self-signed cert with a configuration file

the following Bash command writes a multiline string to a new file

Instructions:
  1. Update the output directory
  2. Update the configuration manually with your values. See [openSSl](https://www.openssl.org/) library for details

Modern standards (e.g., RFC 2818 for HTTPS) prioritize SAN( [alt_names] ) over CN (commonName)f or identity verification, as SAN supports multiple identifiers and is less ambiguous.


 - DNS or dNSName (DNS Name), which represents a fully qualified domain name (FQDN) 
   -  e.g.,  `DNS.1=www.example.org`
 - otherName (Tag [0], OtherName)  for NPI or Taxid (Most public CAs (e.g., DigiCert, Entrust) may not support custom otherName  :-()

   - define an OID
   - NPI Value as string
   -  otherName.1: Specifies the NPI with the format OID;TYPE:VALUE.
        - e.g. : `otherName.1 = 2.16.840.1.113883.4.6;UTF8:1234567890`
- URI or uniformResourceIdentifier (Tag [6], IA5String):  for FHIR resource 
  -  e.g., `URI.1 = https://example.org/fhir/Practitioner/123`


In [1]:
%%bash
# Define a directory variable
cat << 'EOF' | tee /tmp/vars.sh  #bash commands do not edit
DIR_PATH="example_org_cert"  #update directory with your value
EOF

DIR_PATH="example_org_cert"  #update directory with your value


In [ ]:
%%bash
# =================== bash commands do not edit =======================
# Source the script to load the variable
source /tmp/vars.sh
# Create the directory if it doesn't exist
mkdir -p "$DIR_PATH"  # -p ensures no error if directory already exists

# Use here-document to write to a file in the specified directory
cat << EOF| tee "$DIR_PATH/cert.config"  # Write to newfile.txt in the directory
# ===========Configuration for healthcare certificate with NPI and FHIR endpoint ===========
# =================== update configuration manually with your values =======================
[req]
default_bit = 4096
distinguished_name = req_distinguished_name
prompt = no
x509_extensions = v3_ca

# Subject details
[req_distinguished_name]
countryName             = US
stateOrProvinceName     = California
localityName            = Sausalito
organizationName        = Example Organization
commonName              = John Hancock, MD
emailAddress            = jhancock@example.org

[v3_ca]
basicConstraints = CA:TRUE
keyUsage=nonRepudiation, digitalSignature, keyEncipherment

# SAN extension
subjectAltName = @alt_names

# SAN entries for FHIR and NPI
[alt_names]
DNS.1 = www.example.org
otherName.1 = 2.16.840.1.113883.4.6;UTF8:9941339108
URI.1 = https://example.org/fhir/Practitioner/123
EOF
# don't edit the previous line

### Step 2: Generate the public and private keys and cert using the openSSL command line tool.

- Saved in the $DIR_PATH folder
- For the purpose of this example display the keys (normally would never share the private key)


In [ ]:
%%bash
# Source the script to load the variable
source /tmp/vars.sh
# generate the private 
openssl genrsa -out "$DIR_PATH/private-key.pem" 3072
openssl rsa -in "$DIR_PATH/private-key.pem" -pubout -out "$DIR_PATH/public-key.pem"
openssl req -new -x509 -key "$DIR_PATH/private-key.pem" -outform DER -out "$DIR_PATH/cert.der" -days 720 -config "$DIR_PATH/cert.config"
echo  "********* normally you would never share this! ***********"
cat "$DIR_PATH/private-key.pem"
echo  "********* you only share this! ***********"
cat "$DIR_PATH/public-key.pem"


#### Show the Certificate in DER Format  

In [22]:
%%bash
# Source the script to load the variable
source /tmp/vars.sh
openssl x509 -in "$DIR_PATH/cert.der" -inform DER -text

Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number:
            3c:0d:0c:be:38:ca:19:4d:ee:2f:3f:6b:36:44:1f:56:43:a6:91:25
        Signature Algorithm: sha256WithRSAEncryption
        Issuer: C = US, ST = California, L = Sausalito, O = Example Organization, CN = "John Hancock, MD", emailAddress = jhancock@example.org
        Validity
            Not Before: Jun 19 21:19:01 2025 GMT
            Not After : Jun  9 21:19:01 2027 GMT
        Subject: C = US, ST = California, L = Sausalito, O = Example Organization, CN = "John Hancock, MD", emailAddress = jhancock@example.org
        Subject Public Key Info:
            Public Key Algorithm: rsaEncryption
                Public-Key: (3072 bit)
                Modulus:
                    00:b4:2e:a1:c3:16:2c:c9:1a:da:32:03:af:13:d4:
                    c6:db:3b:0f:80:c9:5e:53:5e:01:04:e4:70:e8:6a:
                    28:64:97:13:c6:2d:48:2a:60:55:32:e7:e4:39:8a:
                    9c:2a:7e:d4:33:ad:90:5c:4a:e5:a6:07:a3:

##### Show the Certificate in PEM format

In [ ]:
%%bash
# Source the script to load the variable
source /tmp/vars.sh
openssl x509 -in "$DIR_PATH/cert.der" -inform DER -outform PEM -out "$DIR_PATH/cert.pem"
cat "$DIR_PATH/cert.pem"